<h1>导入数据</h1>

In [600]:
import matplotlib.pyplot as plt
import pandas as pd
import time
from datetime import datetime
from tqdm import tqdm
from time import strftime, localtime
from scipy import stats
from scipy.spatial import distance
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.utils.class_weight import compute_class_weight
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score
from sklearn.model_selection import GridSearchCV
from lightgbm import LGBMClassifier
from joblib import dump, load
from tensorflow.keras.backend import epsilon
from itertools import combinations
from imblearn.over_sampling import SMOTE

<h1>数据处理</h1>

<h2>数据预处理</h2>

<h3>conf_info数据预处理</h3>

In [601]:
cons_info = pd.read_csv("../data/训练组_比特币挖矿_档案明细（20211220）.csv", index_col=0, quotechar='"', encoding="gbk")
cons_info

,ID,ELEC_TYPE_NAME,VOLT_NAME,PRC_NAME,CONTRACT_CAP,RUN_CAP,SHIFT_NO,BUILD_DATE,CANCEL_DATE,CHK_CYCLE,LAST_CHK_DATE,TMP_NAME,TMP_DATE,IS_FLAG
,,,,,,,,,,,,,,
1,329465205,乡村居民生活用电,交流220V,居民合表电价(不满1千伏）,4.0,4.0,1.0,2005/6/18,NaN,240.0,2005/6/18,非临时用电,NaN,0
2,329465245,乡村居民生活用电,交流220V,居民合表电价(不满1千伏）,4.0,4.0,1.0,2005/6/18,NaN,240.0,2005/6/18,非临时用电,NaN,0
3,329465357,乡村居民生活用电,交流220V,居民合表电价(不满1千伏）,8.0,8.0,1.0,2005/6/18,NaN,240.0,2005/6/18,非临时用电,NaN,0
4,329465360,乡村居民生活用电,交流220V,居民合表电价(不满1千伏）,2.0,2.0,1.0,2005/6/18,NaN,240.0,2005/6/18,非临时用电,NaN,0
5,329465439,乡村居民生活用电,交流220V,居民合表电价(不满1千伏）,8.0,8.0,1.0,2005/6/18,NaN,240.0,2005/6/18,非临时用电,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8836,2862711473,乡村居民生活用电,交流380V,居民合表电价(不满1千伏）,39.0,39.0,NaN,2019/12/2 16:12:00,NaN,120.0,2019/12/2 16:12:00,非临时用电,NaN,0
8837,2864274650,乡村居民生活用电,交流220V,居民合表电价(不满1千伏）,13.0,13.0,NaN,2019/12/3 16:33:00,NaN,120.0,2019/12/3 16:33:00,非临时用电,NaN,0
8838,2872313739,乡村居民生活用电,交流380V,居民合表电价(不满1千伏）,39.0,39.0,NaN,2019/12/18 14:42:00,NaN,120.0,2019/12/18 14:42:00,非临时用电,NaN,0


In [602]:
cons_info = cons_info.set_index("ID").drop(columns=["CANCEL_DATE", "TMP_DATE"])
cons_info

,ELEC_TYPE_NAME,VOLT_NAME,PRC_NAME,CONTRACT_CAP,RUN_CAP,SHIFT_NO,BUILD_DATE,CHK_CYCLE,LAST_CHK_DATE,TMP_NAME,IS_FLAG
ID,,,,,,,,,,,
329465205,乡村居民生活用电,交流220V,居民合表电价(不满1千伏）,4.0,4.0,1.0,2005/6/18,240.0,2005/6/18,非临时用电,0
329465245,乡村居民生活用电,交流220V,居民合表电价(不满1千伏）,4.0,4.0,1.0,2005/6/18,240.0,2005/6/18,非临时用电,0
329465357,乡村居民生活用电,交流220V,居民合表电价(不满1千伏）,8.0,8.0,1.0,2005/6/18,240.0,2005/6/18,非临时用电,0
329465360,乡村居民生活用电,交流220V,居民合表电价(不满1千伏）,2.0,2.0,1.0,2005/6/18,240.0,2005/6/18,非临时用电,0
329465439,乡村居民生活用电,交流220V,居民合表电价(不满1千伏）,8.0,8.0,1.0,2005/6/18,240.0,2005/6/18,非临时用电,0
...,...,...,...,...,...,...,...,...,...,...,...
2862711473,乡村居民生活用电,交流380V,居民合表电价(不满1千伏）,39.0,39.0,NaN,2019/12/2 16:12:00,120.0,2019/12/2 16:12:00,非临时用电,0
2864274650,乡村居民生活用电,交流220V,居民合表电价(不满1千伏）,13.0,13.0,NaN,2019/12/3 16:33:00,120.0,2019/12/3 16:33:00,非临时用电,0
2872313739,乡村居民生活用电,交流380V,居民合表电价(不满1千伏）,39.0,39.0,NaN,2019/12/18 14:42:00,120.0,2019/12/18 14:42:00,非临时用电,0


In [603]:
cons_info["BUILD_DATE"] = pd.to_datetime(cons_info["BUILD_DATE"])
cons_info["LAST_CHK_DATE"] = pd.to_datetime(cons_info["LAST_CHK_DATE"])
cons_info

,ELEC_TYPE_NAME,VOLT_NAME,PRC_NAME,CONTRACT_CAP,RUN_CAP,SHIFT_NO,BUILD_DATE,CHK_CYCLE,LAST_CHK_DATE,TMP_NAME,IS_FLAG
ID,,,,,,,,,,,
329465205,乡村居民生活用电,交流220V,居民合表电价(不满1千伏）,4.0,4.0,1.0,2005-06-18 00:00:00,240.0,2005-06-18 00:00:00,非临时用电,0
329465245,乡村居民生活用电,交流220V,居民合表电价(不满1千伏）,4.0,4.0,1.0,2005-06-18 00:00:00,240.0,2005-06-18 00:00:00,非临时用电,0
329465357,乡村居民生活用电,交流220V,居民合表电价(不满1千伏）,8.0,8.0,1.0,2005-06-18 00:00:00,240.0,2005-06-18 00:00:00,非临时用电,0
329465360,乡村居民生活用电,交流220V,居民合表电价(不满1千伏）,2.0,2.0,1.0,2005-06-18 00:00:00,240.0,2005-06-18 00:00:00,非临时用电,0
329465439,乡村居民生活用电,交流220V,居民合表电价(不满1千伏）,8.0,8.0,1.0,2005-06-18 00:00:00,240.0,2005-06-18 00:00:00,非临时用电,0
...,...,...,...,...,...,...,...,...,...,...,...
2862711473,乡村居民生活用电,交流380V,居民合表电价(不满1千伏）,39.0,39.0,NaN,2019-12-02 16:12:00,120.0,2019-12-02 16:12:00,非临时用电,0
2864274650,乡村居民生活用电,交流220V,居民合表电价(不满1千伏）,13.0,13.0,NaN,2019-12-03 16:33:00,120.0,2019-12-03 16:33:00,非临时用电,0
2872313739,乡村居民生活用电,交流380V,居民合表电价(不满1千伏）,39.0,39.0,NaN,2019-12-18 14:42:00,120.0,2019-12-18 14:42:00,非临时用电,0


In [604]:
cons_info["SHIFT_NO"] = cons_info["SHIFT_NO"].fillna(0)
cons_info

,ELEC_TYPE_NAME,VOLT_NAME,PRC_NAME,CONTRACT_CAP,RUN_CAP,SHIFT_NO,BUILD_DATE,CHK_CYCLE,LAST_CHK_DATE,TMP_NAME,IS_FLAG
ID,,,,,,,,,,,
329465205,乡村居民生活用电,交流220V,居民合表电价(不满1千伏）,4.0,4.0,1.0,2005-06-18 00:00:00,240.0,2005-06-18 00:00:00,非临时用电,0
329465245,乡村居民生活用电,交流220V,居民合表电价(不满1千伏）,4.0,4.0,1.0,2005-06-18 00:00:00,240.0,2005-06-18 00:00:00,非临时用电,0
329465357,乡村居民生活用电,交流220V,居民合表电价(不满1千伏）,8.0,8.0,1.0,2005-06-18 00:00:00,240.0,2005-06-18 00:00:00,非临时用电,0
329465360,乡村居民生活用电,交流220V,居民合表电价(不满1千伏）,2.0,2.0,1.0,2005-06-18 00:00:00,240.0,2005-06-18 00:00:00,非临时用电,0
329465439,乡村居民生活用电,交流220V,居民合表电价(不满1千伏）,8.0,8.0,1.0,2005-06-18 00:00:00,240.0,2005-06-18 00:00:00,非临时用电,0
...,...,...,...,...,...,...,...,...,...,...,...
2862711473,乡村居民生活用电,交流380V,居民合表电价(不满1千伏）,39.0,39.0,0.0,2019-12-02 16:12:00,120.0,2019-12-02 16:12:00,非临时用电,0
2864274650,乡村居民生活用电,交流220V,居民合表电价(不满1千伏）,13.0,13.0,0.0,2019-12-03 16:33:00,120.0,2019-12-03 16:33:00,非临时用电,0
2872313739,乡村居民生活用电,交流380V,居民合表电价(不满1千伏）,39.0,39.0,0.0,2019-12-18 14:42:00,120.0,2019-12-18 14:42:00,非临时用电,0


In [605]:
cons_info["now"] = "2021-12-31"
cons_info["now"] = pd.to_datetime(cons_info["now"])
cons_info

,ELEC_TYPE_NAME,VOLT_NAME,PRC_NAME,CONTRACT_CAP,RUN_CAP,SHIFT_NO,BUILD_DATE,CHK_CYCLE,LAST_CHK_DATE,TMP_NAME,IS_FLAG,now
ID,,,,,,,,,,,,
329465205,乡村居民生活用电,交流220V,居民合表电价(不满1千伏）,4.0,4.0,1.0,2005-06-18 00:00:00,240.0,2005-06-18 00:00:00,非临时用电,0,2021-12-31
329465245,乡村居民生活用电,交流220V,居民合表电价(不满1千伏）,4.0,4.0,1.0,2005-06-18 00:00:00,240.0,2005-06-18 00:00:00,非临时用电,0,2021-12-31
329465357,乡村居民生活用电,交流220V,居民合表电价(不满1千伏）,8.0,8.0,1.0,2005-06-18 00:00:00,240.0,2005-06-18 00:00:00,非临时用电,0,2021-12-31
329465360,乡村居民生活用电,交流220V,居民合表电价(不满1千伏）,2.0,2.0,1.0,2005-06-18 00:00:00,240.0,2005-06-18 00:00:00,非临时用电,0,2021-12-31
329465439,乡村居民生活用电,交流220V,居民合表电价(不满1千伏）,8.0,8.0,1.0,2005-06-18 00:00:00,240.0,2005-06-18 00:00:00,非临时用电,0,2021-12-31
...,...,...,...,...,...,...,...,...,...,...,...,...
2862711473,乡村居民生活用电,交流380V,居民合表电价(不满1千伏）,39.0,39.0,0.0,2019-12-02 16:12:00,120.0,2019-12-02 16:12:00,非临时用电,0,2021-12-31
2864274650,乡村居民生活用电,交流220V,居民合表电价(不满1千伏）,13.0,13.0,0.0,2019-12-03 16:33:00,120.0,2019-12-03 16:33:00,非临时用电,0,2021-12-31
2872313739,乡村居民生活用电,交流380V,居民合表电价(不满1千伏）,39.0,39.0,0.0,2019-12-18 14:42:00,120.0,2019-12-18 14:42:00,非临时用电,0,2021-12-31


In [606]:
cons_info["live_days"] = (cons_info["now"] - cons_info["BUILD_DATE"]).map(lambda x: x.days)
cons_info["check_days"] = (cons_info["now"] - cons_info["LAST_CHK_DATE"]).map(lambda x: x.days)
cons_info

,ELEC_TYPE_NAME,VOLT_NAME,PRC_NAME,CONTRACT_CAP,RUN_CAP,SHIFT_NO,BUILD_DATE,CHK_CYCLE,LAST_CHK_DATE,TMP_NAME,IS_FLAG,now,live_days,check_days
ID,,,,,,,,,,,,,,
329465205,乡村居民生活用电,交流220V,居民合表电价(不满1千伏）,4.0,4.0,1.0,2005-06-18 00:00:00,240.0,2005-06-18 00:00:00,非临时用电,0,2021-12-31,6040,6040
329465245,乡村居民生活用电,交流220V,居民合表电价(不满1千伏）,4.0,4.0,1.0,2005-06-18 00:00:00,240.0,2005-06-18 00:00:00,非临时用电,0,2021-12-31,6040,6040
329465357,乡村居民生活用电,交流220V,居民合表电价(不满1千伏）,8.0,8.0,1.0,2005-06-18 00:00:00,240.0,2005-06-18 00:00:00,非临时用电,0,2021-12-31,6040,6040
329465360,乡村居民生活用电,交流220V,居民合表电价(不满1千伏）,2.0,2.0,1.0,2005-06-18 00:00:00,240.0,2005-06-18 00:00:00,非临时用电,0,2021-12-31,6040,6040
329465439,乡村居民生活用电,交流220V,居民合表电价(不满1千伏）,8.0,8.0,1.0,2005-06-18 00:00:00,240.0,2005-06-18 00:00:00,非临时用电,0,2021-12-31,6040,6040
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2862711473,乡村居民生活用电,交流380V,居民合表电价(不满1千伏）,39.0,39.0,0.0,2019-12-02 16:12:00,120.0,2019-12-02 16:12:00,非临时用电,0,2021-12-31,759,759
2864274650,乡村居民生活用电,交流220V,居民合表电价(不满1千伏）,13.0,13.0,0.0,2019-12-03 16:33:00,120.0,2019-12-03 16:33:00,非临时用电,0,2021-12-31,758,758
2872313739,乡村居民生活用电,交流380V,居民合表电价(不满1千伏）,39.0,39.0,0.0,2019-12-18 14:42:00,120.0,2019-12-18 14:42:00,非临时用电,0,2021-12-31,743,743


In [607]:
cons_info = cons_info.drop(columns=["BUILD_DATE", "LAST_CHK_DATE", "now"])
cons_info

,ELEC_TYPE_NAME,VOLT_NAME,PRC_NAME,CONTRACT_CAP,RUN_CAP,SHIFT_NO,CHK_CYCLE,TMP_NAME,IS_FLAG,live_days,check_days
ID,,,,,,,,,,,
329465205,乡村居民生活用电,交流220V,居民合表电价(不满1千伏）,4.0,4.0,1.0,240.0,非临时用电,0,6040,6040
329465245,乡村居民生活用电,交流220V,居民合表电价(不满1千伏）,4.0,4.0,1.0,240.0,非临时用电,0,6040,6040
329465357,乡村居民生活用电,交流220V,居民合表电价(不满1千伏）,8.0,8.0,1.0,240.0,非临时用电,0,6040,6040
329465360,乡村居民生活用电,交流220V,居民合表电价(不满1千伏）,2.0,2.0,1.0,240.0,非临时用电,0,6040,6040
329465439,乡村居民生活用电,交流220V,居民合表电价(不满1千伏）,8.0,8.0,1.0,240.0,非临时用电,0,6040,6040
...,...,...,...,...,...,...,...,...,...,...,...
2862711473,乡村居民生活用电,交流380V,居民合表电价(不满1千伏）,39.0,39.0,0.0,120.0,非临时用电,0,759,759
2864274650,乡村居民生活用电,交流220V,居民合表电价(不满1千伏）,13.0,13.0,0.0,120.0,非临时用电,0,758,758
2872313739,乡村居民生活用电,交流380V,居民合表电价(不满1千伏）,39.0,39.0,0.0,120.0,非临时用电,0,743,743


In [608]:
cons_info.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8840 entries, 329465205 to 2529558569
Data columns (total 11 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   ELEC_TYPE_NAME  8840 non-null   object 
 1   VOLT_NAME       8840 non-null   object 
 2   PRC_NAME        8840 non-null   object 
 3   CONTRACT_CAP    8840 non-null   float64
 4   RUN_CAP         8840 non-null   float64
 5   SHIFT_NO        8840 non-null   float64
 6   CHK_CYCLE       8838 non-null   float64
 7   TMP_NAME        8840 non-null   object 
 8   IS_FLAG         8840 non-null   int64  
 9   live_days       8840 non-null   int64  
 10  check_days      8840 non-null   int64  
dtypes: float64(4), int64(3), object(4)
memory usage: 828.8+ KB


In [609]:
cons_info = pd.get_dummies(cons_info)
cons_info

,CONTRACT_CAP,RUN_CAP,SHIFT_NO,CHK_CYCLE,IS_FLAG,live_days,check_days,ELEC_TYPE_NAME_乡村居民生活用电,ELEC_TYPE_NAME_商业用电,ELEC_TYPE_NAME_城镇居民生活用电,...,ELEC_TYPE_NAME_居民生活用电,ELEC_TYPE_NAME_普通工业,ELEC_TYPE_NAME_非居民照明,ELEC_TYPE_NAME_非工业,VOLT_NAME_交流10kV,VOLT_NAME_交流220V,VOLT_NAME_交流380V,PRC_NAME_居民合表电价(1-10千伏）,PRC_NAME_居民合表电价(不满1千伏）,TMP_NAME_非临时用电
ID,,,,,,,,,,,,,,,,,,,,,
329465205,4.0,4.0,1.0,240.0,0,6040,6040,1,0,0,...,0,0,0,0,0,1,0,0,1,1
329465245,4.0,4.0,1.0,240.0,0,6040,6040,1,0,0,...,0,0,0,0,0,1,0,0,1,1
329465357,8.0,8.0,1.0,240.0,0,6040,6040,1,0,0,...,0,0,0,0,0,1,0,0,1,1
329465360,2.0,2.0,1.0,240.0,0,6040,6040,1,0,0,...,0,0,0,0,0,1,0,0,1,1
329465439,8.0,8.0,1.0,240.0,0,6040,6040,1,0,0,...,0,0,0,0,0,1,0,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2862711473,39.0,39.0,0.0,120.0,0,759,759,1,0,0,...,0,0,0,0,0,0,1,0,1,1
2864274650,13.0,13.0,0.0,120.0,0,758,758,1,0,0,...,0,0,0,0,0,1,0,0,1,1
2872313739,39.0,39.0,0.0,120.0,0,743,743,1,0,0,...,0,0,0,0,0,0,1,0,1,1


In [610]:
cons_info["CHK_CYCLE"] = cons_info["CHK_CYCLE"].fillna(cons_info["CHK_CYCLE"].median())
cons_info.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8840 entries, 329465205 to 2529558569
Data columns (total 21 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   CONTRACT_CAP                8840 non-null   float64
 1   RUN_CAP                     8840 non-null   float64
 2   SHIFT_NO                    8840 non-null   float64
 3   CHK_CYCLE                   8840 non-null   float64
 4   IS_FLAG                     8840 non-null   int64  
 5   live_days                   8840 non-null   int64  
 6   check_days                  8840 non-null   int64  
 7   ELEC_TYPE_NAME_乡村居民生活用电     8840 non-null   uint8  
 8   ELEC_TYPE_NAME_商业用电         8840 non-null   uint8  
 9   ELEC_TYPE_NAME_城镇居民生活用电     8840 non-null   uint8  
 10  ELEC_TYPE_NAME_学校教学和学生生活用电  8840 non-null   uint8  
 11  ELEC_TYPE_NAME_居民生活用电       8840 non-null   uint8  
 12  ELEC_TYPE_NAME_普通工业         8840 non-null   uint8  
 13  ELEC_TYPE_NAME_非居民照

<h3>elec_month数据预处理</h3>

In [611]:
cons_info["pq_z_mean"] = 0
cons_info["pq_z_median"] = 0
cons_info["pq_z_std"] = 0
cons_info["pq_z_std/mean"] = 0
cons_info["pq_z_entropy"] = 0
cons_info["pq_fg_distance"] = 0
cons_info["pq_gp_diatance"] = 0
cons_info["pq_fp_distance"] = 0
cons_info["pq_fg_cosine"] = 0
cons_info["pq_gp_cosine"] = 0
cons_info["pq_fp_cosine"] = 0
cons_info["pq_fz_cosine"] = 0
cons_info["pq_gz_cosine"] = 0
cons_info["pq_pz_cosine"] = 0
cons_info["variation_month"] = 0
cons_info["pq_z_mean_4500"] = 0
cons_info["variation_month_0085"] = 0

cons_info["pq_z_max"] = 0
cons_info["pq_z_min"] = 0

cons_info["entropy_month"] = 0
cons_info["std_month"] = 0

cons_info["diff_month"] = 0
cons_info["devide_month"] = 0

In [612]:
def entropy(data, bins: int = 10) -> float:
    """
    计算一个序列的熵\n
    """
    nums = np.histogram(data, bins=bins)[0]
    probs = nums / sum(nums)
    return stats.entropy(probs)

In [613]:
def entropy_df(df: pd.DataFrame, bins: int = 10) -> float:
    """
    计算一段时间子电量的熵\n
    """
    entropys = df.apply(lambda x: entropy(x), axis=1)
    return entropys.mean()

In [614]:
def std_df(df: pd.DataFrame) -> float:
    """
    计算一段时间子电量的标准差\n
    """
    return df.std(axis=1).mean()

In [615]:
def cosine(a, b) -> float:
    """
    计算余弦距离\n
    """
    if np.linalg.norm(a) != 0 and np.linalg.norm(b) != 0:
        return distance.cosine(a, b)
    elif np.linalg.norm(a) == 0 and np.linalg.norm(b) == 0:
        return 0
    else:
        return 1

In [616]:
def diff_median(df: pd.DataFrame) -> float:
    """
    计算一个df的两列数值差的中位数\n
    """
    results = []
    for i, j in combinations(list(range(df.shape[1])), r=2):
        result = (df.iloc[:, i] - df.iloc[:, j]).apply(lambda x: abs(x)).median()
        results.append(result)
    return sum(results)

In [617]:
def devide_median(df: pd.DataFrame) -> float:
    """
    计算两个列的相除的中位数\n
    """
    results = []
    for i, j in combinations(list(range(df.shape[1])), r=2):
        result = (df.iloc[:, 0] / (df.iloc[:, 1] + epsilon())).median()
        results.append(result)
    return sum(results)

In [618]:
def variation(df: pd.DataFrame) -> float:
    """
    计算变异系数\n
    """
    tt =  (epsilon() + df.std(axis=1)) / (epsilon() + df.mean(axis=1))
    tt[tt.apply(lambda x: np.isinf(x))] = 0
    return tt.mean(axis=0)

In [619]:
elec_month = pd.read_csv("../data/训练组_比特币挖矿_月用电明细（20211217）.csv")
elec_month

,id,ym,pq_f,pq_g,pq_p,pq_z
0,329769614,202001,615,161,0,776
1,329769614,202002,492,154,-1,645
2,329769614,202003,457,114,1,572
3,329769614,202004,522,171,0,693
4,329769614,202005,519,181,0,700
...,...,...,...,...,...,...
194475,2878334859,202106,107,67,-1,173
194476,2878334859,202107,170,112,1,283
194477,2878334859,202108,102,67,0,169
194478,2878334859,202109,112,65,-1,176


In [620]:
elec_month["ym"] = elec_month["ym"].astype(str)
elec_month

,id,ym,pq_f,pq_g,pq_p,pq_z
0,329769614,202001,615,161,0,776
1,329769614,202002,492,154,-1,645
2,329769614,202003,457,114,1,572
3,329769614,202004,522,171,0,693
4,329769614,202005,519,181,0,700
...,...,...,...,...,...,...
194475,2878334859,202106,107,67,-1,173
194476,2878334859,202107,170,112,1,283
194477,2878334859,202108,102,67,0,169
194478,2878334859,202109,112,65,-1,176


In [621]:
for cons_id in tqdm(cons_info.index):
    df = elec_month[elec_month["id"] == cons_id]
    if df.shape[0] == 0:
        continue
    df = df.drop(columns=["id"]).sort_values("ym").set_index("ym")
    df = df.fillna(method="ffill").fillna(method="bfill").fillna(0)
    cons_info.loc[cons_id, "pq_z_mean"] = df["pq_z"].mean()
    cons_info.loc[cons_id, "pq_z_median"] = df["pq_z"].median()
    cons_info.loc[cons_id, "pq_z_std"] = df["pq_z"].std()
    cons_info.loc[cons_id, "pq_z_std/mean"] = (cons_info.loc[cons_id, "pq_z_std"] + epsilon()) / (cons_info.loc[cons_id, "pq_z_mean"] + epsilon())
    cons_info.loc[cons_id, "pq_z_entropy"] = entropy(df["pq_z"])
    
    cons_info.loc[cons_id, "pq_fg_distance"] = distance.euclidean(df["pq_f"], df["pq_g"])
    cons_info.loc[cons_id, "pq_gp_diatance"] = distance.euclidean(df["pq_g"], df["pq_p"])
    cons_info.loc[cons_id, "pq_fp_distance"] = distance.euclidean(df["pq_f"], df["pq_p"])
    
    cons_info.loc[cons_id, "pq_fg_cosine"] = cosine(df["pq_f"], df["pq_g"])
    cons_info.loc[cons_id, "pq_gp_cosine"] = cosine(df["pq_g"], df["pq_p"])
    cons_info.loc[cons_id, "pq_fp_cosine"] = cosine(df["pq_f"], df["pq_p"])
    cons_info.loc[cons_id, "pq_fz_cosine"] = cosine(df["pq_f"], df["pq_z"])
    cons_info.loc[cons_id, "pq_gz_cosine"] = cosine(df["pq_g"], df["pq_z"])
    cons_info.loc[cons_id, "pq_pz_cosine"] = cosine(df["pq_p"], df["pq_z"])
    
    cons_info.loc[cons_id, "variation_month"] = variation(df[["pq_f", "pq_g", "pq_p"]])
    cons_info.loc[cons_id, "pq_z_mean_4500"] = (cons_info.loc[cons_id, "pq_z_mean"] > 4500).astype(float)
    cons_info.loc[cons_id, "variation_month_0085"] = (cons_info.loc[cons_id, "variation_month"] < 0.085).astype(float)
    
    cons_info.loc[cons_id, "pq_z_max"] = df["pq_z"].max()
    cons_info.loc[cons_id, "pq_z_min"] = df["pq_z"].min()
    
    cons_info.loc[cons_id, "entropy_month"] = entropy_df(df[["pq_f", "pq_g", "pq_p"]])
    cons_info.loc[cons_id, "std_month"] = std_df(df[["pq_f", "pq_g", "pq_p"]])
    
    cons_info.loc[cons_id, "diff_month"] = diff_median(df[["pq_f", "pq_g", "pq_p"]])
    cons_info.loc[cons_id, "devide_month"] = devide_median(df[["pq_f", "pq_g", "pq_p"]])  
    
    cons_info.loc[cons_id, "pq_z/cap"] = (df["pq_z"].max() + epsilon()) + (cons_info.loc[cons_id, "CONTRACT_CAP"] + epsilon())
    cons_info.loc[cons_id, "pq_g/cap"] = (df["pq_g"].max() + epsilon()) + (cons_info.loc[cons_id, "CONTRACT_CAP"] + epsilon())
    
    cons_info.loc[cons_id, "max_idx"] = df["pq_z"].idxmax()
    cons_info.loc[cons_id, "pq_p-g_std"] = (df["pq_p"] - df["pq_g"]).std()
    cons_info.loc[cons_id, "pq_f-g_std"] = (df["pq_f"] - df["pq_g"]).std()

    
cons_info

100%|████████████████████████████████████████████████████████████████████████████████| 8840/8840 [03:12<00:00, 45.93it/s]


,CONTRACT_CAP,RUN_CAP,SHIFT_NO,CHK_CYCLE,IS_FLAG,live_days,check_days,ELEC_TYPE_NAME_乡村居民生活用电,ELEC_TYPE_NAME_商业用电,ELEC_TYPE_NAME_城镇居民生活用电,...,pq_z_min,entropy_month,std_month,diff_month,devide_month,pq_z/cap,pq_g/cap,max_idx,pq_p-g_std,pq_f-g_std
ID,,,,,,,,,,,,,,,,,,,,,
329465205,4.0,4.0,1.0,240.0,0,6040,6040,1,0,0,...,218,1.098612,236.742067,915.0,5.338216,1444.0,484.0,202010,136.044683,108.656265
329465245,4.0,4.0,1.0,240.0,0,6040,6040,1,0,0,...,613,1.098612,356.826146,1349.5,6.739272,2267.0,711.0,202010,122.923585,127.838177
329465357,8.0,8.0,1.0,240.0,0,6040,6040,1,0,0,...,1186,0.678523,941.880880,3151.0,0.000000,3409.0,442.0,202010,659.979214,210.043306
329465360,2.0,2.0,1.0,240.0,0,6040,6040,1,0,0,...,553,1.098612,384.279868,1495.5,7.056769,2242.0,645.0,202010,119.248658,127.269085
329465439,8.0,8.0,1.0,240.0,0,6040,6040,1,0,0,...,678,0.636514,681.299561,2319.0,0.000000,2219.0,8.0,202010,273.560036,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2862711473,39.0,39.0,0.0,120.0,0,759,759,1,0,0,...,808,1.098612,532.213416,2065.5,5.876878,3083.0,1068.0,202010,153.487933,156.404467
2864274650,13.0,13.0,0.0,120.0,0,758,758,1,0,0,...,53,1.098612,148.889645,122.0,5.550620,1683.0,523.0,202010,132.124867,191.058614
2872313739,39.0,39.0,0.0,120.0,0,743,743,1,0,0,...,0,0.078869,0.329195,0.0,0.000000,49.0,40.0,202108,0.213201,1.870250


<h3>elec_day数据预处理</h3>

In [622]:
cons_info["kwh_mean"] = 0
cons_info["kwh_median"] = 0
cons_info["kwh_std"] = 0
cons_info["kwh_std/mean"] = 0
cons_info["kwh_entropy"] = 0

cons_info["kwh_pap_r23_distance"] = 0
cons_info["kwh_pap_r24_distance"] = 0
cons_info["kwh_pap_r34_distance"] = 0

cons_info["kwh_pap_r23_cosine"] = 0
cons_info["kwh_pap_r24_cosine"] = 0
cons_info["kwh_pap_r34_cosine"] = 0

cons_info["kwh_pap_r1_cosine"] = 0
cons_info["kwh_pap_r2_cosine"] = 0
cons_info["kwh_pap_r3_cosine"] = 0
cons_info["kwh_pap_r4_cosine"] = 0
cons_info["variation_day"] = 0

cons_info["kwh_max"] = 0
cons_info["kwh_min"] = 0

cons_info["entropy_day"] = 0
cons_info["std_day"] = 0

cons_info["diff_day"] = 0
cons_info["devide_day"] = 0

In [623]:
elec_day = pd.read_csv("../data/训练组_比特币挖矿_日用电明细（20211217）.csv")
elec_day

,id,rq,kwh,kwh_rap,kwh_pap_r1,kwh_pap_r2,kwh_pap_r3,kwh_pap_r4
0,329449398,2020-01-22 00:00:00,11.77,0.0,0.0,7.40,0.0,4.36
1,329449398,2020-01-23 00:00:00,16.54,0.0,0.0,8.68,0.0,7.86
2,329449398,2020-01-24 00:00:00,10.86,0.0,0.0,5.51,0.0,5.36
3,329449398,2020-01-25 00:00:00,10.91,0.0,0.0,6.48,0.0,4.43
4,329449398,2020-01-26 00:00:00,10.85,0.0,0.0,6.32,0.0,4.52
...,...,...,...,...,...,...,...,...
954564,2878334859,2021-10-05 00:00:00,7.22,NaN,0.0,4.49,0.0,2.73
954565,2878334859,2021-10-06 00:00:00,7.23,NaN,0.0,4.48,0.0,2.74
954566,2878334859,2021-10-07 00:00:00,7.20,NaN,0.0,4.48,0.0,2.73
954567,2878334859,2021-10-08 00:00:00,7.23,NaN,0.0,4.50,0.0,2.73


In [624]:
elec_day["rq"] = pd.to_datetime(elec_day["rq"])
elec_day

,id,rq,kwh,kwh_rap,kwh_pap_r1,kwh_pap_r2,kwh_pap_r3,kwh_pap_r4
0,329449398,2020-01-22,11.77,0.0,0.0,7.40,0.0,4.36
1,329449398,2020-01-23,16.54,0.0,0.0,8.68,0.0,7.86
2,329449398,2020-01-24,10.86,0.0,0.0,5.51,0.0,5.36
3,329449398,2020-01-25,10.91,0.0,0.0,6.48,0.0,4.43
4,329449398,2020-01-26,10.85,0.0,0.0,6.32,0.0,4.52
...,...,...,...,...,...,...,...,...
954564,2878334859,2021-10-05,7.22,NaN,0.0,4.49,0.0,2.73
954565,2878334859,2021-10-06,7.23,NaN,0.0,4.48,0.0,2.74
954566,2878334859,2021-10-07,7.20,NaN,0.0,4.48,0.0,2.73
954567,2878334859,2021-10-08,7.23,NaN,0.0,4.50,0.0,2.73


In [625]:
for cons_id in tqdm(cons_info.index):
    df = elec_day[elec_day["id"] == cons_id]
    df = df.drop(columns=["id"]).sort_values("rq").set_index("rq")
    df = df.fillna(method="ffill").fillna(method="bfill").fillna(0)
    cons_info.loc[cons_id, "kwh_mean"] = df["kwh"].mean()
    cons_info.loc[cons_id, "kwh_median"] = df["kwh"].median()
    cons_info.loc[cons_id, "kwh_std"] = df["kwh"].std()
    cons_info.loc[cons_id, "kwh_std/mean"] = (cons_info.loc[cons_id, "kwh_std"] + epsilon()) / (cons_info.loc[cons_id, "kwh_mean"] + epsilon())
    cons_info.loc[cons_id, "kwh_entropy"] = entropy(df["kwh"])

    cons_info.loc[cons_id, "kwh_pap_r23_distance"] = distance.euclidean(df["kwh_pap_r2"], df["kwh_pap_r3"])
    cons_info.loc[cons_id, "kwh_pap_r24_distance"] = distance.euclidean(df["kwh_pap_r2"], df["kwh_pap_r4"])
    cons_info.loc[cons_id, "kwh_pap_r34_distance"] = distance.euclidean(df["kwh_pap_r3"], df["kwh_pap_r4"])

    cons_info.loc[cons_id, "kwh_pap_r23_cosine"] = cosine(df["kwh_pap_r2"], df["kwh_pap_r3"])
    cons_info.loc[cons_id, "kwh_pap_r24_cosine"] = cosine(df["kwh_pap_r2"], df["kwh_pap_r4"])
    cons_info.loc[cons_id, "kwh_pap_r34_cosine"] = cosine(df["kwh_pap_r3"], df["kwh_pap_r4"])
    
    cons_info.loc[cons_id, "kwh_pap_r1_cosine"] = cosine(df["kwh_pap_r1"], df["kwh"])
    cons_info.loc[cons_id, "kwh_pap_r2_cosine"] = cosine(df["kwh_pap_r2"], df["kwh"])
    cons_info.loc[cons_id, "kwh_pap_r3_cosine"] = cosine(df["kwh_pap_r3"], df["kwh"])
    cons_info.loc[cons_id, "kwh_pap_r4_cosine"] = cosine(df["kwh_pap_r4"], df["kwh"])
    
    cons_info.loc[cons_id, "variation_day"] = variation(df[["kwh_pap_r2", "kwh_pap_r3", "kwh_pap_r4"]])
    
    cons_info.loc[cons_id, "kwh_max"] = df["kwh"].max()
    cons_info.loc[cons_id, "kwh_min"] = df["kwh"].min()
    
    cons_info.loc[cons_id, "entropy_day"] = entropy_df(df[["kwh_pap_r2", "kwh_pap_r3", "kwh_pap_r4"]])
    cons_info.loc[cons_id, "std_day"] = std_df(df[["kwh_pap_r2", "kwh_pap_r3", "kwh_pap_r4"]])
    
    cons_info.loc[cons_id, "diff_day"] = diff_median(df[["kwh_pap_r2", "kwh_pap_r3", "kwh_pap_r4"]])
    cons_info.loc[cons_id, "devide_day"] = devide_median(df[["kwh_pap_r2", "kwh_pap_r3", "kwh_pap_r4"]])
    
    
cons_info

100%|████████████████████████████████████████████████████████████████████████████████| 8840/8840 [04:53<00:00, 30.08it/s]


,CONTRACT_CAP,RUN_CAP,SHIFT_NO,CHK_CYCLE,IS_FLAG,live_days,check_days,ELEC_TYPE_NAME_乡村居民生活用电,ELEC_TYPE_NAME_商业用电,ELEC_TYPE_NAME_城镇居民生活用电,...,kwh_pap_r2_cosine,kwh_pap_r3_cosine,kwh_pap_r4_cosine,variation_day,kwh_max,kwh_min,entropy_day,std_day,diff_day,devide_day
ID,,,,,,,,,,,,,,,,,,,,,
329465205,4.0,4.0,1.0,240.0,0,6040,6040,1,0,0,...,0.013884,1.0,0.048088,1.039291,45.71,0.66,1.060104,6.192106,20.875,3.454500e+08
329465245,4.0,4.0,1.0,240.0,0,6040,6040,1,0,0,...,0.004351,1.0,0.019658,1.045283,54.73,17.88,1.098612,12.109005,47.185,7.074000e+08
329465357,8.0,8.0,1.0,240.0,0,6040,6040,1,0,0,...,0.001687,1.0,0.024876,1.236592,79.75,26.31,1.090055,22.801373,88.530,1.329000e+09
329465360,2.0,2.0,1.0,240.0,0,6040,6040,1,0,0,...,0.002899,1.0,0.017558,1.103814,74.44,14.16,1.098612,12.624684,49.825,7.327500e+08
329465439,8.0,8.0,1.0,240.0,0,6040,6040,1,0,0,...,0.003703,1.0,0.023050,1.094278,52.60,16.35,1.098612,12.514458,47.980,7.183500e+08
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2862711473,39.0,39.0,0.0,120.0,0,759,759,1,0,0,...,0.005853,1.0,0.019615,0.962473,74.60,13.68,1.085776,12.263838,42.910,6.667500e+08
2864274650,13.0,13.0,0.0,120.0,0,758,758,1,0,0,...,0.002854,1.0,0.013023,0.965715,42.91,1.50,1.085776,4.196080,4.035,5.565000e+07
2872313739,39.0,39.0,0.0,120.0,0,743,743,1,0,0,...,0.000000,0.0,0.000000,1.000000,0.00,0.00,0.000000,0.000000,0.000,0.000000e+00


<h2>数据集划分</h2>

In [626]:
x = cons_info.drop(columns=["IS_FLAG"])
y = cons_info["IS_FLAG"]
print(x.shape, y.shape)

(8840, 70) (8840,)


<h2>标准化</h2>

In [627]:
ss = StandardScaler()
x = ss.fit_transform(x)
x

array([[-0.31260749, -0.31917163,  1.05812375, ...,  0.05176978,
         0.04269137,  0.55764591],
       [-0.31260749, -0.31917163,  1.05812375, ...,  0.30898424,
         0.34491997,  1.84521864],
       [-0.2464131 , -0.25086739,  1.05812375, ...,  0.77379383,
         0.81985884,  4.05645004],
       ...,
       [ 0.2665934 ,  0.27849043, -0.86811471, ..., -0.21740823,
        -0.19710423, -0.671231  ],
       [ 0.2665934 ,  0.27849043, -0.86811471, ..., -0.09737511,
        -0.10543626, -0.23794917],
       [ 0.11765603,  0.1248059 , -0.86811471, ...,  0.14671492,
         0.17812562,  1.06883309]])

<h1>做出来一个模型</h1>

In [628]:
smo = SMOTE(random_state=42, n_jobs=-1)
x, y = smo.fit_sample(x, y)

In [629]:
model = LGBMClassifier(
    objective="binary", 
    n_jobs=-1
)

In [630]:
cv = GridSearchCV(
    estimator=model,
    param_grid={
        "n_estimators": list(range(224, 235, 2)),
        "num_leaves": list(range(4, 7, 1)),
        "max_depth": [-1],
        "subsample_for_bin": list(range(10000, 50000, 10000))
    },
    scoring="f1",
    n_jobs=-1,
    iid=False,
    cv=10,
    verbose=1
)

In [631]:
try:
    model = load("../models/best_lightBGM.pkl")
    print("n_estimators", model.n_estimators)
    print("num_leaves", model.num_leaves)
    print("max_depth", model.max_depth)
    print("subsample_for_bin", model.subsample_for_bin)
    f1 = f1_score(y, model.predict(x))
except FileNotFoundError as e:
    cv.fit(x, y)
    model = cv.best_estimator_
    print(cv.best_params_)
    f1 = cv.best_score_
    model.fit(x, y)
    dump(model, "../models/best_lightBGM.pkl")

n_estimators 224
num_leaves 6
max_depth -1
subsample_for_bin 10000


In [632]:
f1

1.0

In [633]:
accuracy_score(y, model.predict(x))

1.0

In [634]:
roc_auc_score(y, model.predict(x))

1.0

<h1>处理测试数据</h1>

<h2>数据预处理</h2>

<h3>档案数据预处理</h3>

In [635]:
cons_info_test = pd.read_csv("../data/B组_比特币挖矿_档案明细（20220313）.csv", index_col=0, quotechar='"', encoding="utf8")
cons_info_test = cons_info_test.rename(columns={"Unnamed: 3": "PRC_NAME"})
cons_info_test = cons_info_test.rename(columns={k: k.upper() for k in cons_info_test.columns})
cons_info_test

,ELEC_TYPE_NAME,VOLT_NAME,PRC_NAME,CONTRACT_CAP,RUN_CAP,SHIFT_NO,BUILD_DATE,CANCEL_DATE,CHK_CYCLE,LAST_CHK_DATE,TMP_NAME,TMP_DATE
id,,,,,,,,,,,,
183817867,城镇居民生活用电,交流220V,居民生活<1kV(合表),2.0,2.0,1.0,2000-01-01 00:00:00,NaN,24,2000-01-01 00:00:00,非临时用电,NaN
183819011,城镇居民生活用电,交流220V,居民生活<1kV(合表),2.0,2.0,1.0,2000-01-01 00:00:00,NaN,24,2000-01-01 00:00:00,非临时用电,NaN
183819032,城镇居民生活用电,交流220V,居民生活<1kV(合表),2.0,2.0,1.0,2000-01-01 00:00:00,NaN,24,2000-01-01 00:00:00,非临时用电,NaN
183821932,城镇居民生活用电,交流220V,居民生活<1kV(合表),2.0,2.0,1.0,2000-01-01 00:00:00,NaN,24,2000-01-01 00:00:00,非临时用电,NaN
183821933,城镇居民生活用电,交流220V,居民生活<1kV(合表),2.0,2.0,1.0,2000-01-01 00:00:00,NaN,24,2000-01-01 00:00:00,非临时用电,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
2747851717,乡村居民生活用电,交流380V,居民生活<1kV(合表),105.0,105.0,NaN,2019-05-17 16:25:00,NaN,120,2019-05-17 16:25:00,非临时用电,NaN
2748294248,非居民照明,交流220V,居民生活<1kV(合表),8.0,8.0,NaN,2019-05-09 15:25:00,NaN,120,2019-05-09 15:25:00,非临时用电,NaN
2754499432,城镇居民生活用电,交流380V,居民生活<1kV(合表),526.0,526.0,NaN,2019-05-24 09:52:00,NaN,120,2019-05-24 09:52:00,非临时用电,NaN


In [636]:
cons_info_test = cons_info_test.drop(columns=["CANCEL_DATE", "TMP_DATE"])
cons_info_test

,ELEC_TYPE_NAME,VOLT_NAME,PRC_NAME,CONTRACT_CAP,RUN_CAP,SHIFT_NO,BUILD_DATE,CHK_CYCLE,LAST_CHK_DATE,TMP_NAME
id,,,,,,,,,,
183817867,城镇居民生活用电,交流220V,居民生活<1kV(合表),2.0,2.0,1.0,2000-01-01 00:00:00,24,2000-01-01 00:00:00,非临时用电
183819011,城镇居民生活用电,交流220V,居民生活<1kV(合表),2.0,2.0,1.0,2000-01-01 00:00:00,24,2000-01-01 00:00:00,非临时用电
183819032,城镇居民生活用电,交流220V,居民生活<1kV(合表),2.0,2.0,1.0,2000-01-01 00:00:00,24,2000-01-01 00:00:00,非临时用电
183821932,城镇居民生活用电,交流220V,居民生活<1kV(合表),2.0,2.0,1.0,2000-01-01 00:00:00,24,2000-01-01 00:00:00,非临时用电
183821933,城镇居民生活用电,交流220V,居民生活<1kV(合表),2.0,2.0,1.0,2000-01-01 00:00:00,24,2000-01-01 00:00:00,非临时用电
...,...,...,...,...,...,...,...,...,...,...
2747851717,乡村居民生活用电,交流380V,居民生活<1kV(合表),105.0,105.0,NaN,2019-05-17 16:25:00,120,2019-05-17 16:25:00,非临时用电
2748294248,非居民照明,交流220V,居民生活<1kV(合表),8.0,8.0,NaN,2019-05-09 15:25:00,120,2019-05-09 15:25:00,非临时用电
2754499432,城镇居民生活用电,交流380V,居民生活<1kV(合表),526.0,526.0,NaN,2019-05-24 09:52:00,120,2019-05-24 09:52:00,非临时用电


In [637]:
cons_info_test["BUILD_DATE"] = pd.to_datetime(cons_info_test["BUILD_DATE"])
cons_info_test["LAST_CHK_DATE"] = pd.to_datetime(cons_info_test["LAST_CHK_DATE"])
cons_info_test

,ELEC_TYPE_NAME,VOLT_NAME,PRC_NAME,CONTRACT_CAP,RUN_CAP,SHIFT_NO,BUILD_DATE,CHK_CYCLE,LAST_CHK_DATE,TMP_NAME
id,,,,,,,,,,
183817867,城镇居民生活用电,交流220V,居民生活<1kV(合表),2.0,2.0,1.0,2000-01-01 00:00:00,24,2000-01-01 00:00:00,非临时用电
183819011,城镇居民生活用电,交流220V,居民生活<1kV(合表),2.0,2.0,1.0,2000-01-01 00:00:00,24,2000-01-01 00:00:00,非临时用电
183819032,城镇居民生活用电,交流220V,居民生活<1kV(合表),2.0,2.0,1.0,2000-01-01 00:00:00,24,2000-01-01 00:00:00,非临时用电
183821932,城镇居民生活用电,交流220V,居民生活<1kV(合表),2.0,2.0,1.0,2000-01-01 00:00:00,24,2000-01-01 00:00:00,非临时用电
183821933,城镇居民生活用电,交流220V,居民生活<1kV(合表),2.0,2.0,1.0,2000-01-01 00:00:00,24,2000-01-01 00:00:00,非临时用电
...,...,...,...,...,...,...,...,...,...,...
2747851717,乡村居民生活用电,交流380V,居民生活<1kV(合表),105.0,105.0,NaN,2019-05-17 16:25:00,120,2019-05-17 16:25:00,非临时用电
2748294248,非居民照明,交流220V,居民生活<1kV(合表),8.0,8.0,NaN,2019-05-09 15:25:00,120,2019-05-09 15:25:00,非临时用电
2754499432,城镇居民生活用电,交流380V,居民生活<1kV(合表),526.0,526.0,NaN,2019-05-24 09:52:00,120,2019-05-24 09:52:00,非临时用电


In [638]:
cons_info_test["SHIFT_NO"] = cons_info_test["SHIFT_NO"].fillna(0)
cons_info_test

,ELEC_TYPE_NAME,VOLT_NAME,PRC_NAME,CONTRACT_CAP,RUN_CAP,SHIFT_NO,BUILD_DATE,CHK_CYCLE,LAST_CHK_DATE,TMP_NAME
id,,,,,,,,,,
183817867,城镇居民生活用电,交流220V,居民生活<1kV(合表),2.0,2.0,1.0,2000-01-01 00:00:00,24,2000-01-01 00:00:00,非临时用电
183819011,城镇居民生活用电,交流220V,居民生活<1kV(合表),2.0,2.0,1.0,2000-01-01 00:00:00,24,2000-01-01 00:00:00,非临时用电
183819032,城镇居民生活用电,交流220V,居民生活<1kV(合表),2.0,2.0,1.0,2000-01-01 00:00:00,24,2000-01-01 00:00:00,非临时用电
183821932,城镇居民生活用电,交流220V,居民生活<1kV(合表),2.0,2.0,1.0,2000-01-01 00:00:00,24,2000-01-01 00:00:00,非临时用电
183821933,城镇居民生活用电,交流220V,居民生活<1kV(合表),2.0,2.0,1.0,2000-01-01 00:00:00,24,2000-01-01 00:00:00,非临时用电
...,...,...,...,...,...,...,...,...,...,...
2747851717,乡村居民生活用电,交流380V,居民生活<1kV(合表),105.0,105.0,0.0,2019-05-17 16:25:00,120,2019-05-17 16:25:00,非临时用电
2748294248,非居民照明,交流220V,居民生活<1kV(合表),8.0,8.0,0.0,2019-05-09 15:25:00,120,2019-05-09 15:25:00,非临时用电
2754499432,城镇居民生活用电,交流380V,居民生活<1kV(合表),526.0,526.0,0.0,2019-05-24 09:52:00,120,2019-05-24 09:52:00,非临时用电


In [639]:
cons_info_test["now"] = "2021-12-31"
cons_info_test["now"] = pd.to_datetime(cons_info_test["now"])
cons_info_test

,ELEC_TYPE_NAME,VOLT_NAME,PRC_NAME,CONTRACT_CAP,RUN_CAP,SHIFT_NO,BUILD_DATE,CHK_CYCLE,LAST_CHK_DATE,TMP_NAME,now
id,,,,,,,,,,,
183817867,城镇居民生活用电,交流220V,居民生活<1kV(合表),2.0,2.0,1.0,2000-01-01 00:00:00,24,2000-01-01 00:00:00,非临时用电,2021-12-31
183819011,城镇居民生活用电,交流220V,居民生活<1kV(合表),2.0,2.0,1.0,2000-01-01 00:00:00,24,2000-01-01 00:00:00,非临时用电,2021-12-31
183819032,城镇居民生活用电,交流220V,居民生活<1kV(合表),2.0,2.0,1.0,2000-01-01 00:00:00,24,2000-01-01 00:00:00,非临时用电,2021-12-31
183821932,城镇居民生活用电,交流220V,居民生活<1kV(合表),2.0,2.0,1.0,2000-01-01 00:00:00,24,2000-01-01 00:00:00,非临时用电,2021-12-31
183821933,城镇居民生活用电,交流220V,居民生活<1kV(合表),2.0,2.0,1.0,2000-01-01 00:00:00,24,2000-01-01 00:00:00,非临时用电,2021-12-31
...,...,...,...,...,...,...,...,...,...,...,...
2747851717,乡村居民生活用电,交流380V,居民生活<1kV(合表),105.0,105.0,0.0,2019-05-17 16:25:00,120,2019-05-17 16:25:00,非临时用电,2021-12-31
2748294248,非居民照明,交流220V,居民生活<1kV(合表),8.0,8.0,0.0,2019-05-09 15:25:00,120,2019-05-09 15:25:00,非临时用电,2021-12-31
2754499432,城镇居民生活用电,交流380V,居民生活<1kV(合表),526.0,526.0,0.0,2019-05-24 09:52:00,120,2019-05-24 09:52:00,非临时用电,2021-12-31


In [640]:
cons_info_test["live_days"] = (cons_info_test["now"] - cons_info_test["BUILD_DATE"]).map(lambda x: x.days)
cons_info_test["check_days"] = (cons_info_test["now"] - cons_info_test["LAST_CHK_DATE"]).map(lambda x: x.days)
cons_info_test

,ELEC_TYPE_NAME,VOLT_NAME,PRC_NAME,CONTRACT_CAP,RUN_CAP,SHIFT_NO,BUILD_DATE,CHK_CYCLE,LAST_CHK_DATE,TMP_NAME,now,live_days,check_days
id,,,,,,,,,,,,,
183817867,城镇居民生活用电,交流220V,居民生活<1kV(合表),2.0,2.0,1.0,2000-01-01 00:00:00,24,2000-01-01 00:00:00,非临时用电,2021-12-31,8035,8035
183819011,城镇居民生活用电,交流220V,居民生活<1kV(合表),2.0,2.0,1.0,2000-01-01 00:00:00,24,2000-01-01 00:00:00,非临时用电,2021-12-31,8035,8035
183819032,城镇居民生活用电,交流220V,居民生活<1kV(合表),2.0,2.0,1.0,2000-01-01 00:00:00,24,2000-01-01 00:00:00,非临时用电,2021-12-31,8035,8035
183821932,城镇居民生活用电,交流220V,居民生活<1kV(合表),2.0,2.0,1.0,2000-01-01 00:00:00,24,2000-01-01 00:00:00,非临时用电,2021-12-31,8035,8035
183821933,城镇居民生活用电,交流220V,居民生活<1kV(合表),2.0,2.0,1.0,2000-01-01 00:00:00,24,2000-01-01 00:00:00,非临时用电,2021-12-31,8035,8035
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2747851717,乡村居民生活用电,交流380V,居民生活<1kV(合表),105.0,105.0,0.0,2019-05-17 16:25:00,120,2019-05-17 16:25:00,非临时用电,2021-12-31,958,958
2748294248,非居民照明,交流220V,居民生活<1kV(合表),8.0,8.0,0.0,2019-05-09 15:25:00,120,2019-05-09 15:25:00,非临时用电,2021-12-31,966,966
2754499432,城镇居民生活用电,交流380V,居民生活<1kV(合表),526.0,526.0,0.0,2019-05-24 09:52:00,120,2019-05-24 09:52:00,非临时用电,2021-12-31,951,951


In [641]:
cons_info_test = cons_info_test.drop(columns=["BUILD_DATE", "LAST_CHK_DATE", "now"])
cons_info_test

,ELEC_TYPE_NAME,VOLT_NAME,PRC_NAME,CONTRACT_CAP,RUN_CAP,SHIFT_NO,CHK_CYCLE,TMP_NAME,live_days,check_days
id,,,,,,,,,,
183817867,城镇居民生活用电,交流220V,居民生活<1kV(合表),2.0,2.0,1.0,24,非临时用电,8035,8035
183819011,城镇居民生活用电,交流220V,居民生活<1kV(合表),2.0,2.0,1.0,24,非临时用电,8035,8035
183819032,城镇居民生活用电,交流220V,居民生活<1kV(合表),2.0,2.0,1.0,24,非临时用电,8035,8035
183821932,城镇居民生活用电,交流220V,居民生活<1kV(合表),2.0,2.0,1.0,24,非临时用电,8035,8035
183821933,城镇居民生活用电,交流220V,居民生活<1kV(合表),2.0,2.0,1.0,24,非临时用电,8035,8035
...,...,...,...,...,...,...,...,...,...,...
2747851717,乡村居民生活用电,交流380V,居民生活<1kV(合表),105.0,105.0,0.0,120,非临时用电,958,958
2748294248,非居民照明,交流220V,居民生活<1kV(合表),8.0,8.0,0.0,120,非临时用电,966,966
2754499432,城镇居民生活用电,交流380V,居民生活<1kV(合表),526.0,526.0,0.0,120,非临时用电,951,951


In [642]:
cons_info_test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 22218 entries, 183817867 to 107237458
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   ELEC_TYPE_NAME  22218 non-null  object 
 1   VOLT_NAME       22218 non-null  object 
 2   PRC_NAME        22218 non-null  object 
 3   CONTRACT_CAP    22218 non-null  float64
 4   RUN_CAP         22218 non-null  float64
 5   SHIFT_NO        22218 non-null  float64
 6   CHK_CYCLE       22218 non-null  int64  
 7   TMP_NAME        22218 non-null  object 
 8   live_days       22218 non-null  int64  
 9   check_days      22218 non-null  int64  
dtypes: float64(3), int64(3), object(4)
memory usage: 1.9+ MB


In [643]:
cons_info_test = pd.get_dummies(cons_info_test)
cons_info_test

,CONTRACT_CAP,RUN_CAP,SHIFT_NO,CHK_CYCLE,live_days,check_days,ELEC_TYPE_NAME_乡村居民生活用电,ELEC_TYPE_NAME_农业生产用电,ELEC_TYPE_NAME_商业用电,ELEC_TYPE_NAME_城镇居民生活用电,...,ELEC_TYPE_NAME_非居民照明,ELEC_TYPE_NAME_非工业,VOLT_NAME_交流10kV,VOLT_NAME_交流220V,VOLT_NAME_交流35kV,VOLT_NAME_交流380V,PRC_NAME_居民合表电价(1-10千伏）,PRC_NAME_居民生活1_10kV(合表),PRC_NAME_居民生活<1kV(合表),TMP_NAME_非临时用电
id,,,,,,,,,,,,,,,,,,,,,
183817867,2.0,2.0,1.0,24,8035,8035,0,0,0,1,...,0,0,0,1,0,0,0,0,1,1
183819011,2.0,2.0,1.0,24,8035,8035,0,0,0,1,...,0,0,0,1,0,0,0,0,1,1
183819032,2.0,2.0,1.0,24,8035,8035,0,0,0,1,...,0,0,0,1,0,0,0,0,1,1
183821932,2.0,2.0,1.0,24,8035,8035,0,0,0,1,...,0,0,0,1,0,0,0,0,1,1
183821933,2.0,2.0,1.0,24,8035,8035,0,0,0,1,...,0,0,0,1,0,0,0,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2747851717,105.0,105.0,0.0,120,958,958,1,0,0,0,...,0,0,0,0,0,1,0,0,1,1
2748294248,8.0,8.0,0.0,120,966,966,0,0,0,0,...,1,0,0,1,0,0,0,0,1,1
2754499432,526.0,526.0,0.0,120,951,951,0,0,0,1,...,0,0,0,0,0,1,0,0,1,1


In [644]:
cons_info_test["CHK_CYCLE"] = cons_info_test["CHK_CYCLE"].fillna(cons_info_test["CHK_CYCLE"].median())
cons_info_test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 22218 entries, 183817867 to 107237458
Data columns (total 25 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   CONTRACT_CAP                22218 non-null  float64
 1   RUN_CAP                     22218 non-null  float64
 2   SHIFT_NO                    22218 non-null  float64
 3   CHK_CYCLE                   22218 non-null  int64  
 4   live_days                   22218 non-null  int64  
 5   check_days                  22218 non-null  int64  
 6   ELEC_TYPE_NAME_乡村居民生活用电     22218 non-null  uint8  
 7   ELEC_TYPE_NAME_农业生产用电       22218 non-null  uint8  
 8   ELEC_TYPE_NAME_商业用电         22218 non-null  uint8  
 9   ELEC_TYPE_NAME_城镇居民生活用电     22218 non-null  uint8  
 10  ELEC_TYPE_NAME_大工业用电        22218 non-null  uint8  
 11  ELEC_TYPE_NAME_学校教学和学生生活用电  22218 non-null  uint8  
 12  ELEC_TYPE_NAME_居民生活用电       22218 non-null  uint8  
 13  ELEC_TYPE_NAME_普通工业

In [645]:
cons_info_test["PRC_NAME_居民合表电价(1-10千伏）"] = cons_info_test["PRC_NAME_居民合表电价(1-10千伏）"] + cons_info_test["PRC_NAME_居民生活1_10kV(合表)"]
cons_info_test = cons_info_test.drop(columns=["PRC_NAME_居民生活1_10kV(合表)"])
cons_info_test

,CONTRACT_CAP,RUN_CAP,SHIFT_NO,CHK_CYCLE,live_days,check_days,ELEC_TYPE_NAME_乡村居民生活用电,ELEC_TYPE_NAME_农业生产用电,ELEC_TYPE_NAME_商业用电,ELEC_TYPE_NAME_城镇居民生活用电,...,ELEC_TYPE_NAME_趸售居民生活用电,ELEC_TYPE_NAME_非居民照明,ELEC_TYPE_NAME_非工业,VOLT_NAME_交流10kV,VOLT_NAME_交流220V,VOLT_NAME_交流35kV,VOLT_NAME_交流380V,PRC_NAME_居民合表电价(1-10千伏）,PRC_NAME_居民生活<1kV(合表),TMP_NAME_非临时用电
id,,,,,,,,,,,,,,,,,,,,,
183817867,2.0,2.0,1.0,24,8035,8035,0,0,0,1,...,0,0,0,0,1,0,0,0,1,1
183819011,2.0,2.0,1.0,24,8035,8035,0,0,0,1,...,0,0,0,0,1,0,0,0,1,1
183819032,2.0,2.0,1.0,24,8035,8035,0,0,0,1,...,0,0,0,0,1,0,0,0,1,1
183821932,2.0,2.0,1.0,24,8035,8035,0,0,0,1,...,0,0,0,0,1,0,0,0,1,1
183821933,2.0,2.0,1.0,24,8035,8035,0,0,0,1,...,0,0,0,0,1,0,0,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2747851717,105.0,105.0,0.0,120,958,958,1,0,0,0,...,0,0,0,0,0,0,1,0,1,1
2748294248,8.0,8.0,0.0,120,966,966,0,0,0,0,...,0,1,0,0,1,0,0,0,1,1
2754499432,526.0,526.0,0.0,120,951,951,0,0,0,1,...,0,0,0,0,0,0,1,0,1,1


In [646]:
cons_info_test = cons_info_test.rename(columns={"PRC_NAME_居民生活<1kV(合表)": "PRC_NAME_居民合表电价(不满1千伏）"})
cons_info_test

,CONTRACT_CAP,RUN_CAP,SHIFT_NO,CHK_CYCLE,live_days,check_days,ELEC_TYPE_NAME_乡村居民生活用电,ELEC_TYPE_NAME_农业生产用电,ELEC_TYPE_NAME_商业用电,ELEC_TYPE_NAME_城镇居民生活用电,...,ELEC_TYPE_NAME_趸售居民生活用电,ELEC_TYPE_NAME_非居民照明,ELEC_TYPE_NAME_非工业,VOLT_NAME_交流10kV,VOLT_NAME_交流220V,VOLT_NAME_交流35kV,VOLT_NAME_交流380V,PRC_NAME_居民合表电价(1-10千伏）,PRC_NAME_居民合表电价(不满1千伏）,TMP_NAME_非临时用电
id,,,,,,,,,,,,,,,,,,,,,
183817867,2.0,2.0,1.0,24,8035,8035,0,0,0,1,...,0,0,0,0,1,0,0,0,1,1
183819011,2.0,2.0,1.0,24,8035,8035,0,0,0,1,...,0,0,0,0,1,0,0,0,1,1
183819032,2.0,2.0,1.0,24,8035,8035,0,0,0,1,...,0,0,0,0,1,0,0,0,1,1
183821932,2.0,2.0,1.0,24,8035,8035,0,0,0,1,...,0,0,0,0,1,0,0,0,1,1
183821933,2.0,2.0,1.0,24,8035,8035,0,0,0,1,...,0,0,0,0,1,0,0,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2747851717,105.0,105.0,0.0,120,958,958,1,0,0,0,...,0,0,0,0,0,0,1,0,1,1
2748294248,8.0,8.0,0.0,120,966,966,0,0,0,0,...,0,1,0,0,1,0,0,0,1,1
2754499432,526.0,526.0,0.0,120,951,951,0,0,0,1,...,0,0,0,0,0,0,1,0,1,1


In [647]:
if "VOLT_NAME_交流6kV" in cons_info_test.columns:
    cons_info_test["VOLT_NAME_交流10kV"] = cons_info_test["VOLT_NAME_交流10kV"] + cons_info_test["VOLT_NAME_交流6kV"]
    cons_info_test = cons_info_test.drop(columns=["VOLT_NAME_交流6kV"])
cons_info_test

,CONTRACT_CAP,RUN_CAP,SHIFT_NO,CHK_CYCLE,live_days,check_days,ELEC_TYPE_NAME_乡村居民生活用电,ELEC_TYPE_NAME_农业生产用电,ELEC_TYPE_NAME_商业用电,ELEC_TYPE_NAME_城镇居民生活用电,...,ELEC_TYPE_NAME_趸售居民生活用电,ELEC_TYPE_NAME_非居民照明,ELEC_TYPE_NAME_非工业,VOLT_NAME_交流10kV,VOLT_NAME_交流220V,VOLT_NAME_交流35kV,VOLT_NAME_交流380V,PRC_NAME_居民合表电价(1-10千伏）,PRC_NAME_居民合表电价(不满1千伏）,TMP_NAME_非临时用电
id,,,,,,,,,,,,,,,,,,,,,
183817867,2.0,2.0,1.0,24,8035,8035,0,0,0,1,...,0,0,0,0,1,0,0,0,1,1
183819011,2.0,2.0,1.0,24,8035,8035,0,0,0,1,...,0,0,0,0,1,0,0,0,1,1
183819032,2.0,2.0,1.0,24,8035,8035,0,0,0,1,...,0,0,0,0,1,0,0,0,1,1
183821932,2.0,2.0,1.0,24,8035,8035,0,0,0,1,...,0,0,0,0,1,0,0,0,1,1
183821933,2.0,2.0,1.0,24,8035,8035,0,0,0,1,...,0,0,0,0,1,0,0,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2747851717,105.0,105.0,0.0,120,958,958,1,0,0,0,...,0,0,0,0,0,0,1,0,1,1
2748294248,8.0,8.0,0.0,120,966,966,0,0,0,0,...,0,1,0,0,1,0,0,0,1,1
2754499432,526.0,526.0,0.0,120,951,951,0,0,0,1,...,0,0,0,0,0,0,1,0,1,1


In [648]:
cons_info_test["ELEC_TYPE_NAME_普通工业"] = cons_info_test["ELEC_TYPE_NAME_普通工业"] + cons_info_test["ELEC_TYPE_NAME_大工业用电"]
cons_info_test = cons_info_test.drop(columns=["ELEC_TYPE_NAME_大工业用电"])
cons_info_test["ELEC_TYPE_NAME_非工业"] = cons_info_test["ELEC_TYPE_NAME_非工业"] + cons_info_test["ELEC_TYPE_NAME_农业生产用电"]
cons_info_test = cons_info_test.drop(columns=["ELEC_TYPE_NAME_农业生产用电"])
cons_info_test

,CONTRACT_CAP,RUN_CAP,SHIFT_NO,CHK_CYCLE,live_days,check_days,ELEC_TYPE_NAME_乡村居民生活用电,ELEC_TYPE_NAME_商业用电,ELEC_TYPE_NAME_城镇居民生活用电,ELEC_TYPE_NAME_学校教学和学生生活用电,...,ELEC_TYPE_NAME_趸售居民生活用电,ELEC_TYPE_NAME_非居民照明,ELEC_TYPE_NAME_非工业,VOLT_NAME_交流10kV,VOLT_NAME_交流220V,VOLT_NAME_交流35kV,VOLT_NAME_交流380V,PRC_NAME_居民合表电价(1-10千伏）,PRC_NAME_居民合表电价(不满1千伏）,TMP_NAME_非临时用电
id,,,,,,,,,,,,,,,,,,,,,
183817867,2.0,2.0,1.0,24,8035,8035,0,0,1,0,...,0,0,0,0,1,0,0,0,1,1
183819011,2.0,2.0,1.0,24,8035,8035,0,0,1,0,...,0,0,0,0,1,0,0,0,1,1
183819032,2.0,2.0,1.0,24,8035,8035,0,0,1,0,...,0,0,0,0,1,0,0,0,1,1
183821932,2.0,2.0,1.0,24,8035,8035,0,0,1,0,...,0,0,0,0,1,0,0,0,1,1
183821933,2.0,2.0,1.0,24,8035,8035,0,0,1,0,...,0,0,0,0,1,0,0,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2747851717,105.0,105.0,0.0,120,958,958,1,0,0,0,...,0,0,0,0,0,0,1,0,1,1
2748294248,8.0,8.0,0.0,120,966,966,0,0,0,0,...,0,1,0,0,1,0,0,0,1,1
2754499432,526.0,526.0,0.0,120,951,951,0,0,1,0,...,0,0,0,0,0,0,1,0,1,1


<h3>月电量处理</h3>

In [649]:
cons_info_test["pq_z_mean"] = 0
cons_info_test["pq_z_median"] = 0
cons_info_test["pq_z_std"] = 0
cons_info_test["pq_z_std/mean"] = 0
cons_info_test["pq_z_entropy"] = 0
cons_info_test["pq_fg_distance"] = 0
cons_info_test["pq_gp_diatance"] = 0
cons_info_test["pq_fp_distance"] = 0
cons_info_test["pq_fg_cosine"] = 0
cons_info_test["pq_gp_cosine"] = 0
cons_info_test["pq_fp_cosine"] = 0
cons_info_test["pq_fz_cosine"] = 0
cons_info_test["pq_gz_cosine"] = 0
cons_info_test["pq_pz_cosine"] = 0
cons_info_test["variation_month"] = 0
cons_info_test["pq_z_mean_4500"] = 0
cons_info_test["variation_month_0085"] = 0

cons_info_test["pq_z_max"] = 0
cons_info_test["pq_z_min"] = 0

cons_info_test["entropy_month"] = 0
cons_info_test["std_month"] = 0

cons_info_test["diff_month"] = 0
cons_info_test["devide_month"] = 0

In [650]:
elec_month_test = pd.read_csv("../data/B组_比特币挖矿_月用电明细（20220313）.csv")
elec_month_test

,id,ym,pq_f,pq_g,pq_p,pq_z
0,183779561,202001,0,0,5,5
1,183779561,202002,0,0,5,5
2,183779561,202003,0,0,4,4
3,183779561,202004,0,0,4,4
4,183779561,202005,0,0,5,5
...,...,...,...,...,...,...
488791,2871808629,202107,2309,1998,2279,6586
488792,2871808629,202108,2525,2039,2493,7057
488793,2871808629,202109,2301,1812,2297,6410
488794,2871808629,202110,2167,1701,2155,6023


In [651]:
elec_month_test["ym"] = elec_month_test["ym"].astype(str)
elec_month_test

,id,ym,pq_f,pq_g,pq_p,pq_z
0,183779561,202001,0,0,5,5
1,183779561,202002,0,0,5,5
2,183779561,202003,0,0,4,4
3,183779561,202004,0,0,4,4
4,183779561,202005,0,0,5,5
...,...,...,...,...,...,...
488791,2871808629,202107,2309,1998,2279,6586
488792,2871808629,202108,2525,2039,2493,7057
488793,2871808629,202109,2301,1812,2297,6410
488794,2871808629,202110,2167,1701,2155,6023


In [652]:
for cons_id in tqdm(cons_info_test.index):
    df = elec_month_test[elec_month_test["id"] == cons_id]
    df = df.drop(columns=["id"]).sort_values("ym").set_index("ym")
    df = df.fillna(method="ffill").fillna(method="bfill").fillna(0)
    cons_info_test.loc[cons_id, "pq_z_mean"] = df["pq_z"].mean()
    cons_info_test.loc[cons_id, "pq_z_median"] = df["pq_z"].median()
    cons_info_test.loc[cons_id, "pq_z_std"] = df["pq_z"].std()
    cons_info_test.loc[cons_id, "pq_z_std/mean"] = (cons_info_test.loc[cons_id, "pq_z_std"] + epsilon()) / (cons_info_test.loc[cons_id, "pq_z_mean"] + epsilon())
    cons_info_test.loc[cons_id, "pq_z_entropy"] = entropy(df["pq_z"])
    
    cons_info_test.loc[cons_id, "pq_fg_distance"] = distance.euclidean(df["pq_f"], df["pq_g"])
    cons_info_test.loc[cons_id, "pq_gp_diatance"] = distance.euclidean(df["pq_g"], df["pq_p"])
    cons_info_test.loc[cons_id, "pq_fp_distance"] = distance.euclidean(df["pq_f"], df["pq_p"])
    
    cons_info_test.loc[cons_id, "pq_fg_cosine"] = cosine(df["pq_f"], df["pq_g"])
    cons_info_test.loc[cons_id, "pq_gp_cosine"] = cosine(df["pq_g"], df["pq_p"])
    cons_info_test.loc[cons_id, "pq_fp_cosine"] = cosine(df["pq_f"], df["pq_p"])
    cons_info_test.loc[cons_id, "pq_fz_cosine"] = cosine(df["pq_f"], df["pq_z"])
    cons_info_test.loc[cons_id, "pq_gz_cosine"] = cosine(df["pq_g"], df["pq_z"])
    cons_info_test.loc[cons_id, "pq_pz_cosine"] = cosine(df["pq_p"], df["pq_z"])
    
    cons_info_test.loc[cons_id, "variation_month"] = variation(df[["pq_f", "pq_g", "pq_p"]])
    cons_info_test.loc[cons_id, "pq_z_mean_4500"] = (cons_info_test.loc[cons_id, "pq_z_mean"] > 4500).astype(float)
    cons_info_test.loc[cons_id, "variation_month_0085"] = (cons_info_test.loc[cons_id, "variation_month"] < 0.085).astype(float)

    cons_info_test.loc[cons_id, "pq_z_max"] = df["pq_z"].max()
    cons_info_test.loc[cons_id, "pq_z_min"] = df["pq_z"].min()
    
    cons_info_test.loc[cons_id, "entropy_month"] = entropy_df(df[["pq_f", "pq_g", "pq_p"]])
    cons_info_test.loc[cons_id, "std_month"] = std_df(df[["pq_f", "pq_g", "pq_p"]])
    
    cons_info_test.loc[cons_id, "diff_month"] = diff_median(df[["pq_f", "pq_g", "pq_p"]])
    cons_info_test.loc[cons_id, "devide_month"] = devide_median(df[["pq_f", "pq_g", "pq_p"]])  
    
    cons_info_test.loc[cons_id, "pq_z/cap"] = (df["pq_z"].max() + epsilon()) + (cons_info_test.loc[cons_id, "CONTRACT_CAP"] + epsilon())
    cons_info_test.loc[cons_id, "pq_g/cap"] = (df["pq_g"].max() + epsilon()) + (cons_info_test.loc[cons_id, "CONTRACT_CAP"] + epsilon())
    
    cons_info_test.loc[cons_id, "max_idx"] = df["pq_z"].idxmax()
    cons_info_test.loc[cons_id, "pq_p-g_std"] = (df["pq_p"] - df["pq_g"]).std()
    cons_info_test.loc[cons_id, "pq_f-g_std"] = (df["pq_f"] - df["pq_g"]).std()
    
    
cons_info_test

100%|██████████████████████████████████████████████████████████████████████████████| 22218/22218 [08:30<00:00, 43.50it/s]


,CONTRACT_CAP,RUN_CAP,SHIFT_NO,CHK_CYCLE,live_days,check_days,ELEC_TYPE_NAME_乡村居民生活用电,ELEC_TYPE_NAME_商业用电,ELEC_TYPE_NAME_城镇居民生活用电,ELEC_TYPE_NAME_学校教学和学生生活用电,...,pq_z_min,entropy_month,std_month,diff_month,devide_month,pq_z/cap,pq_g/cap,max_idx,pq_p-g_std,pq_f-g_std
id,,,,,,,,,,,,,,,,,,,,,
183817867,2.0,2.0,1.0,24,8035,8035,0,0,1,0,...,0,1.048675,14.153462,55.0,4.142857,102.0,44.0,202010,6.884326,2.771609
183819011,2.0,2.0,1.0,24,8035,8035,0,0,1,0,...,22,0.783545,9.802270,32.5,2.837719,93.0,43.0,202109,6.622309,2.700008
183819032,2.0,2.0,1.0,24,8035,8035,0,0,1,0,...,69,1.098612,179.850363,557.5,6.842503,1096.0,493.0,202008,153.849627,75.903276
183821932,2.0,2.0,1.0,24,8035,8035,0,0,1,0,...,27,1.098612,13.073056,51.0,5.875000,59.0,21.0,202010,3.388292,4.847680
183821933,2.0,2.0,1.0,24,8035,8035,0,0,1,0,...,12,1.014594,9.315675,35.0,3.547619,72.0,38.0,202010,7.548544,3.032294
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2747851717,105.0,105.0,0.0,120,958,958,1,0,0,0,...,1185,1.035599,114.286069,431.0,2.261682,6117.0,2426.0,202010,111.950215,136.373977
2748294248,8.0,8.0,0.0,120,966,966,0,0,0,0,...,0,1.027671,25.820417,69.0,5.769886,288.0,136.0,202108,28.625965,23.131705
2754499432,526.0,526.0,0.0,120,951,951,0,0,1,0,...,5824,0.825554,558.053887,1985.0,4.239397,19303.0,5601.0,202010,383.350416,350.132285


<h3>日电量处理</h3>

In [653]:
cons_info_test["kwh_mean"] = 0
cons_info_test["kwh_median"] = 0
cons_info_test["kwh_std"] = 0
cons_info_test["kwh_std/mean"] = 0
cons_info_test["kwh_entropy"] = 0

cons_info_test["kwh_pap_r23_distance"] = 0
cons_info_test["kwh_pap_r24_distance"] = 0
cons_info_test["kwh_pap_r34_distance"] = 0

cons_info_test["kwh_pap_r23_cosine"] = 0
cons_info_test["kwh_pap_r24_cosine"] = 0
cons_info_test["kwh_pap_r34_cosine"] = 0

cons_info_test["kwh_pap_r1_cosine"] = 0
cons_info_test["kwh_pap_r2_cosine"] = 0
cons_info_test["kwh_pap_r3_cosine"] = 0
cons_info_test["kwh_pap_r4_cosine"] = 0
cons_info_test["variation_day"] = 0

cons_info_test["kwh_max"] = 0
cons_info_test["kwh_min"] = 0

cons_info_test["entropy_day"] = 0
cons_info_test["std_day"] = 0

cons_info_test["diff_day"] = 0
cons_info_test["devide_day"] = 0

In [654]:
elec_day_test = pd.read_csv("../data/B组_比特币挖矿_日用电明细（20220313）.csv")
elec_day_test

,id,rq,kwh,kwh_rap,kwh_pap_r1,kwh_pap_r2,kwh_pap_r3,kwh_pap_r4
0,6221183,2020-01-22 00:00:00,0.0,NaN,0.0,0.0,0.0,0.0
1,6221183,2020-01-23 00:00:00,0.0,NaN,0.0,0.0,0.0,0.0
2,6221183,2020-01-24 00:00:00,0.0,NaN,0.0,0.0,0.0,0.0
3,6221183,2020-01-25 00:00:00,0.0,NaN,0.0,0.0,0.0,0.0
4,6221183,2020-01-26 00:00:00,0.0,NaN,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...
2393907,2871808629,2021-10-05 00:00:00,212.0,NaN,0.0,76.8,77.6,57.6
2393908,2871808629,2021-10-06 00:00:00,207.2,NaN,0.0,72.0,75.2,60.8
2393909,2871808629,2021-10-07 00:00:00,208.8,NaN,0.0,73.6,74.4,60.0
2393910,2871808629,2021-10-08 00:00:00,204.8,NaN,0.0,72.8,73.6,58.4


In [655]:
elec_day_test["rq"] = pd.to_datetime(elec_day_test["rq"])
elec_day_test

,id,rq,kwh,kwh_rap,kwh_pap_r1,kwh_pap_r2,kwh_pap_r3,kwh_pap_r4
0,6221183,2020-01-22,0.0,NaN,0.0,0.0,0.0,0.0
1,6221183,2020-01-23,0.0,NaN,0.0,0.0,0.0,0.0
2,6221183,2020-01-24,0.0,NaN,0.0,0.0,0.0,0.0
3,6221183,2020-01-25,0.0,NaN,0.0,0.0,0.0,0.0
4,6221183,2020-01-26,0.0,NaN,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...
2393907,2871808629,2021-10-05,212.0,NaN,0.0,76.8,77.6,57.6
2393908,2871808629,2021-10-06,207.2,NaN,0.0,72.0,75.2,60.8
2393909,2871808629,2021-10-07,208.8,NaN,0.0,73.6,74.4,60.0
2393910,2871808629,2021-10-08,204.8,NaN,0.0,72.8,73.6,58.4


In [656]:
for cons_id in tqdm(cons_info_test.index):
    df = elec_day_test[elec_day_test["id"] == cons_id]
    if df.shape[0] == 0:
        continue
    df = df.drop(columns=["id"]).sort_values("rq").set_index("rq")
    df = df.fillna(method="ffill").fillna(method="bfill").fillna(0)
    cons_info_test.loc[cons_id, "kwh_mean"] = df["kwh"].mean()
    cons_info_test.loc[cons_id, "kwh_median"] = df["kwh"].median()
    cons_info_test.loc[cons_id, "kwh_std"] = df["kwh"].std()
    cons_info_test.loc[cons_id, "kwh_std/mean"] = (cons_info_test.loc[cons_id, "kwh_std"] + epsilon()) / (cons_info_test.loc[cons_id, "kwh_mean"] + epsilon())
    cons_info_test.loc[cons_id, "kwh_entropy"] = entropy(df["kwh"])

    cons_info_test.loc[cons_id, "kwh_pap_r23_distance"] = distance.euclidean(df["kwh_pap_r2"], df["kwh_pap_r3"])
    cons_info_test.loc[cons_id, "kwh_pap_r24_distance"] = distance.euclidean(df["kwh_pap_r2"], df["kwh_pap_r4"])
    cons_info_test.loc[cons_id, "kwh_pap_r34_distance"] = distance.euclidean(df["kwh_pap_r3"], df["kwh_pap_r4"])

    cons_info_test.loc[cons_id, "kwh_pap_r23_cosine"] = cosine(df["kwh_pap_r2"], df["kwh_pap_r3"])
    cons_info_test.loc[cons_id, "kwh_pap_r24_cosine"] = cosine(df["kwh_pap_r2"], df["kwh_pap_r4"])
    cons_info_test.loc[cons_id, "kwh_pap_r34_cosine"] = cosine(df["kwh_pap_r3"], df["kwh_pap_r4"])
    
    cons_info_test.loc[cons_id, "kwh_pap_r1_cosine"] = cosine(df["kwh_pap_r1"], df["kwh"])
    cons_info_test.loc[cons_id, "kwh_pap_r2_cosine"] = cosine(df["kwh_pap_r2"], df["kwh"])
    cons_info_test.loc[cons_id, "kwh_pap_r3_cosine"] = cosine(df["kwh_pap_r3"], df["kwh"])
    cons_info_test.loc[cons_id, "kwh_pap_r4_cosine"] = cosine(df["kwh_pap_r4"], df["kwh"])
    
    cons_info_test.loc[cons_id, "variation_day"] = variation(df[["kwh_pap_r2", "kwh_pap_r3", "kwh_pap_r4"]])
    
    cons_info_test.loc[cons_id, "kwh_max"] = df["kwh"].max()
    cons_info_test.loc[cons_id, "kwh_min"] = df["kwh"].min()
    
    cons_info_test.loc[cons_id, "entropy_day"] = entropy_df(df[["kwh_pap_r2", "kwh_pap_r3", "kwh_pap_r4"]])
    cons_info_test.loc[cons_id, "std_day"] = std_df(df[["kwh_pap_r2", "kwh_pap_r3", "kwh_pap_r4"]])
    
    cons_info_test.loc[cons_id, "diff_day"] = diff_median(df[["kwh_pap_r2", "kwh_pap_r3", "kwh_pap_r4"]])
    cons_info_test.loc[cons_id, "devide_day"] = devide_median(df[["kwh_pap_r2", "kwh_pap_r3", "kwh_pap_r4"]])
    
    
cons_info_test

100%|██████████████████████████████████████████████████████████████████████████████| 22218/22218 [13:13<00:00, 28.01it/s]


,CONTRACT_CAP,RUN_CAP,SHIFT_NO,CHK_CYCLE,live_days,check_days,ELEC_TYPE_NAME_乡村居民生活用电,ELEC_TYPE_NAME_商业用电,ELEC_TYPE_NAME_城镇居民生活用电,ELEC_TYPE_NAME_学校教学和学生生活用电,...,kwh_pap_r2_cosine,kwh_pap_r3_cosine,kwh_pap_r4_cosine,variation_day,kwh_max,kwh_min,entropy_day,std_day,diff_day,devide_day
id,,,,,,,,,,,,,,,,,,,,,
183817867,2.0,2.0,1.0,24,8035,8035,0,0,1,0,...,0.000317,1.000000,0.000539,0.910658,1.67,0.00,0.996889,0.421994,1.860,2.790000e+07
183819011,2.0,2.0,1.0,24,8035,8035,0,0,1,0,...,0.006979,1.000000,0.007745,0.888768,3.54,0.58,0.914629,0.352000,1.090,1.470000e+07
183819032,2.0,2.0,1.0,24,8035,8035,0,0,1,0,...,0.016847,1.000000,0.039670,1.085103,42.84,0.95,1.064383,5.255669,16.760,2.757000e+08
183821932,2.0,2.0,1.0,24,8035,8035,0,0,1,0,...,0.008166,1.000000,0.021650,0.978255,1.79,0.49,1.085776,0.413014,1.590,2.430000e+07
183821933,2.0,2.0,1.0,24,8035,8035,0,0,1,0,...,0.009097,1.000000,0.009531,0.907366,1.62,0.30,0.983088,0.284244,1.085,1.290000e+07
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2747851717,105.0,105.0,0.0,120,958,958,1,0,0,0,...,0.008821,0.004593,0.012625,0.205101,125.10,27.00,0.991645,5.103731,15.600,2.879146e+00
2748294248,8.0,8.0,0.0,120,966,966,0,0,0,0,...,0.050107,1.000000,0.107559,1.000933,11.89,0.00,0.900010,0.829623,1.740,2.850000e+07
2754499432,526.0,526.0,0.0,120,951,951,0,0,1,0,...,0.000824,0.000863,0.003919,0.173091,331.20,201.60,0.824210,16.197571,62.400,3.000000e+00


In [657]:
cons_info_test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 22218 entries, 183817867 to 107237458
Data columns (total 72 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   CONTRACT_CAP                22218 non-null  float64
 1   RUN_CAP                     22218 non-null  float64
 2   SHIFT_NO                    22218 non-null  float64
 3   CHK_CYCLE                   22218 non-null  int64  
 4   live_days                   22218 non-null  int64  
 5   check_days                  22218 non-null  int64  
 6   ELEC_TYPE_NAME_乡村居民生活用电     22218 non-null  uint8  
 7   ELEC_TYPE_NAME_商业用电         22218 non-null  uint8  
 8   ELEC_TYPE_NAME_城镇居民生活用电     22218 non-null  uint8  
 9   ELEC_TYPE_NAME_学校教学和学生生活用电  22218 non-null  uint8  
 10  ELEC_TYPE_NAME_居民生活用电       22218 non-null  uint8  
 11  ELEC_TYPE_NAME_普通工业         22218 non-null  uint8  
 12  ELEC_TYPE_NAME_趸售居民生活用电     22218 non-null  uint8  
 13  ELEC_TYPE_NAME_非居民照

<h2>重新排列测试集</h2>

In [658]:
cons_info_test = cons_info_test.reindex(columns=cons_info.columns).drop(columns=["IS_FLAG"])
cons_info_test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 22218 entries, 183817867 to 107237458
Data columns (total 70 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   CONTRACT_CAP                22218 non-null  float64
 1   RUN_CAP                     22218 non-null  float64
 2   SHIFT_NO                    22218 non-null  float64
 3   CHK_CYCLE                   22218 non-null  int64  
 4   live_days                   22218 non-null  int64  
 5   check_days                  22218 non-null  int64  
 6   ELEC_TYPE_NAME_乡村居民生活用电     22218 non-null  uint8  
 7   ELEC_TYPE_NAME_商业用电         22218 non-null  uint8  
 8   ELEC_TYPE_NAME_城镇居民生活用电     22218 non-null  uint8  
 9   ELEC_TYPE_NAME_学校教学和学生生活用电  22218 non-null  uint8  
 10  ELEC_TYPE_NAME_居民生活用电       22218 non-null  uint8  
 11  ELEC_TYPE_NAME_普通工业         22218 non-null  uint8  
 12  ELEC_TYPE_NAME_非居民照明        22218 non-null  uint8  
 13  ELEC_TYPE_NAME_非工业 

In [659]:
x_test = ss.transform(cons_info_test.values)
x_test.shape

(22218, 70)

In [660]:
x_test

array([[-3.45704680e-01, -3.53323746e-01,  1.05812375e+00, ...,
        -1.99063650e-01, -1.75738010e-01, -5.71981718e-01],
       [-3.45704680e-01, -3.53323746e-01,  1.05812375e+00, ...,
        -2.02106388e-01, -1.84583165e-01, -6.18938370e-01],
       [-3.45704680e-01, -3.53323746e-01,  1.05812375e+00, ...,
         1.10617749e-02, -4.57851898e-03,  3.09522694e-01],
       ...,
       [ 8.32576005e+00,  8.59453106e+00, -8.68114707e-01, ...,
         4.86718944e-01,  5.19697935e-01, -6.71230994e-01],
       [ 8.32576005e+00,  8.59453106e+00, -8.68114707e-01, ...,
         8.59846318e-01,  7.49442218e-01, -6.71230993e-01],
       [-3.45704680e-01, -3.53323746e-01,  1.05812375e+00, ...,
        -1.40649680e-01, -1.21346051e-01, -3.12652939e-01]])

<h2>预测数据</h2>

In [661]:
y_test = model.predict(x_test)
cons_info_test["label"] = y_test
cons_info_test

,CONTRACT_CAP,RUN_CAP,SHIFT_NO,CHK_CYCLE,live_days,check_days,ELEC_TYPE_NAME_乡村居民生活用电,ELEC_TYPE_NAME_商业用电,ELEC_TYPE_NAME_城镇居民生活用电,ELEC_TYPE_NAME_学校教学和学生生活用电,...,kwh_pap_r3_cosine,kwh_pap_r4_cosine,variation_day,kwh_max,kwh_min,entropy_day,std_day,diff_day,devide_day,label
id,,,,,,,,,,,,,,,,,,,,,
183817867,2.0,2.0,1.0,24,8035,8035,0,0,1,0,...,1.000000,0.000539,0.910658,1.67,0.00,0.996889,0.421994,1.860,2.790000e+07,0
183819011,2.0,2.0,1.0,24,8035,8035,0,0,1,0,...,1.000000,0.007745,0.888768,3.54,0.58,0.914629,0.352000,1.090,1.470000e+07,0
183819032,2.0,2.0,1.0,24,8035,8035,0,0,1,0,...,1.000000,0.039670,1.085103,42.84,0.95,1.064383,5.255669,16.760,2.757000e+08,0
183821932,2.0,2.0,1.0,24,8035,8035,0,0,1,0,...,1.000000,0.021650,0.978255,1.79,0.49,1.085776,0.413014,1.590,2.430000e+07,0
183821933,2.0,2.0,1.0,24,8035,8035,0,0,1,0,...,1.000000,0.009531,0.907366,1.62,0.30,0.983088,0.284244,1.085,1.290000e+07,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2747851717,105.0,105.0,0.0,120,958,958,1,0,0,0,...,0.004593,0.012625,0.205101,125.10,27.00,0.991645,5.103731,15.600,2.879146e+00,0
2748294248,8.0,8.0,0.0,120,966,966,0,0,0,0,...,1.000000,0.107559,1.000933,11.89,0.00,0.900010,0.829623,1.740,2.850000e+07,0
2754499432,526.0,526.0,0.0,120,951,951,0,0,1,0,...,0.000863,0.003919,0.173091,331.20,201.60,0.824210,16.197571,62.400,3.000000e+00,0


In [662]:
cons_info_test.index.name = "id"
cons_info_test[["label"]].to_csv("../data/resultB-2.csv", index=True)